In [1]:
# Read data from file, transform to structure and import it to mongo
import json
from pprint import pprint
import numpy as np
import sys
import json
from pygments import highlight, lexers, formatters
from datetime import datetime
import pandas as pd
from pandas import DataFrame
import sqlite3
from sqlite3 import Error
import re
from unidecode import unidecode

dateFormat = "%Y-%m-%d %H:%M:%S.%f"
databaseFile = r"/home/jovyan/work/data/medicines/meds_all_data.db"

In [10]:
class SubjectStudy:
    key: str
    value: str
    unit: str
    full: str
    type: str
    created: str

    def __init__(self):
        self.key = None
        self.value = None
        self.unit = None
        self.full = None
        self.type = None
        self.created = None

    def __str__(self):
        return self.key + " " + self.value + " " + self.unit + " " + self.full + " " + self.type


class HospitalStay:
    def __init__(self):
        self.icd9 = []
        self.icd10 = []
        self.subjectStudies = {}
        self.descriptiveMedicalDatas = {}
        self.diagnosticTests = []
        self.generalUrineTest = {}
        self.elementsOfNonCentrifugedUrine = {}
        self.morphology = {}

    id: int
    unit: str
    doctorId: int
    startDate: str
    endDate: str
    icd9: []
    icd10: []
    created: str
    doctorDiagnosises: str
    subjectStudies: {}
    descriptiveMedicalDatas: {}
    diagnosticTests: []
    generalUrineTest: {}
    elementsOfNonCentrifugedUrine: {}
    morphology: {}


class Patient:
    def __init__(self):
        self.hospitalStay = {}

    id: int
    hospitalStay: {}
    age: int
    sex: str

    def __str__(self):
        return self.id + " " + self.age + " " + self.sex


class DescriptiveMedicalData:
    key: str
    value: str

    def __str__(self):
        return self.key + " " + self.value


class DiagnosticTests:
    key: str
    value: str

    def __str__(self):
        return self.key + " " + self.value


def is_not_blank(s):
    return bool(s and s.strip())


def parseDateTime(inputDateTime):
    if type(inputDateTime) is str:
        return datetime.strptime(inputDateTime, dateFormat)
    else:
        return inputDateTime


def removeDelimiters(inputString):
    if not isinstance(inputString, str):
        return inputString;

    if inputString.startswith('>\''):
        inputString = inputString[2:]
    elif inputString.startswith('>'):
        inputString = inputString[1:]

    if inputString.endswith('\'<'):
        inputString = inputString[:-2]
    elif inputString.endswith('<'):
        inputString = inputString[:-1]

    if inputString.startswith("'"):
        inputString = inputString[1:]
    if inputString.endswith("'"):
        inputString = inputString[:-1]

    return inputString


def toJson(obj):
    formatted_json = json.dumps(obj, default=lambda x: x.__dict__, sort_keys=True, indent=4, ensure_ascii=False)
    colorful_json = highlight(formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter())
    return formatted_json.replace('\\\\x0d\\\\x0a', '')


def readPatientsFromDb(db: str, sql: str, existingPatients: dict):
    recordsFromFile = parseDataFile(db, sql)
    addNewPatients(recordsFromFile, existingPatients)
    return buildPatientsArray(existingPatients)

def processPatientData(patientsDf: DataFrame):
    for i in range(0, patientsDf.shape[1] - 1):
        patientsDf[i] = patientsDf[i].apply(removeDelimiters)
    return patientsDf


def parseDataFile(db: str, sql: str) -> DataFrame:
    conn = sqlite3.connect(db)
    return pd.read_sql_query(sql, conn)

def readPatientsFromFile(path: str, sheetName:str, existingPatients: dict):
    recordsFromFile = parseXlsFile(path, sheetName)
    addNewPatients(recordsFromFile, existingPatients)
    return existingPatients

def parseXlsFile(path, sheetName:str,) -> DataFrame:
    patientsDf = pd.read_excel(path, sheet_name=sheetName, header=None)
    for i in range(0, patientsDf.shape[1] - 1):
        patientsDf[i] = patientsDf[i].apply(removeDelimiters)
    return patientsDf

def addNewPatients(records: DataFrame, existingPatients: dict):
    for index, row in records.iterrows():
        buildPatient(existingPatients, row)


def buildPatient(patients, patientData):
    #  0patientId
    #  1stayId
    #  2unit
    #  3doctorId
    #  4dataType
    #  5dateIn
    #  6dateOut
    #  7age
    #  8gender
    #  9icd9
    #  0icd10
    #  1entryDate
    #  2entryName
    #  3entryText

    if patientData[0] in patients:
        patient = patients[patientData[0]]
    else:
        patient = Patient()
        patient.id = patientData[0]
        patient.age = patientData[7]
        patient.sex = patientData[8]
        patients[patientData[0]] = patient

    if patientData[1] not in patient.hospitalStay:
        hospitalStay = HospitalStay()
        hospitalStay.id = patientData[1]
        hospitalStay.unit = patientData[2]
        hospitalStay.doctorId = patientData[3]
        hospitalStay.startDate = patientData[5]
        hospitalStay.endDate = patientData[6]
        if not pd.isnull(patientData[9]):
            hospitalStay.icd9 = patientData[9].split("<|>")
        if not pd.isnull(patientData[10]):
            hospitalStay.icd10 = patientData[10].split("<|>")
        #        hospitalStay.subjectStudy.append(subjectStudy)
        patient.hospitalStay[patientData[1]] = hospitalStay;

    if patientData[4] == 'Badania diagnostyczne':
        diagnosticTests = DiagnosticTests()
        diagnosticTests.key = patientData[12]
        diagnosticTests.value = patientData[13]
        patient.hospitalStay[patientData[1]].diagnosticTests.append(diagnosticTests);
    elif patientData[4] == 'Wyniki badań lab':
        if not pd.isnull(patientData[13]) and is_not_blank(patientData[13]) and not patientData[13].startswith('COMMENT_HL7'):
            if patientData[12] == 'Badanie ogólne moczu':
                if not patient.hospitalStay[patientData[1]].generalUrineTest or parseDateTime(patientData[11]) > parseDateTime(
                        patient.hospitalStay[patientData[1]].generalUrineTest['created']):
                    patient.hospitalStay[patientData[1]].generalUrineTest['created'] = patientData[11]
                    patient.hospitalStay[patientData[1]].generalUrineTest['value'] = patientData[13]
                    urineTests = parseStringTestsResult(patientData[13], 'omocz_')
                    for urineKey, urineValue in urineTests.items():
                        subjectStudy = SubjectStudy()
                        subjectStudy.key = urineKey
                        subjectStudy.type = patientData[4]
                        subjectStudy.created = patientData[11]
                        subjectStudy.value = urineValue
                        subjectStudy.unit = ''
                        patient.hospitalStay[patientData[1]].subjectStudies[subjectStudy.key] = subjectStudy

            elif patientData[12] == 'Badanie - Elementy  moczu niewirowanego':
                if not patient.hospitalStay[patientData[1]].elementsOfNonCentrifugedUrine or parseDateTime(
                        patientData[11]) > parseDateTime(patient.hospitalStay[patientData[1]].elementsOfNonCentrifugedUrine['created']):
                    patient.hospitalStay[patientData[1]].elementsOfNonCentrifugedUrine['created'] = patientData[11]
                    patient.hospitalStay[patientData[1]].elementsOfNonCentrifugedUrine['value'] = patientData[13]
                    urineTests = parseStringTestsResult(patientData[13], 'nmocz_')
                    for urineKey, urineValue in urineTests.items():
                        subjectStudy = SubjectStudy()
                        subjectStudy.key = urineKey
                        subjectStudy.type = patientData[4]
                        subjectStudy.created = patientData[11]
                        subjectStudy.value = urineValue
                        subjectStudy.unit = ''
                        patient.hospitalStay[patientData[1]].subjectStudies[subjectStudy.key] = subjectStudy
            elif patientData[12] == 'Morfologia':
                if not patient.hospitalStay[patientData[1]].morphology or parseDateTime(patientData[11]) > parseDateTime(
                        patient.hospitalStay[patientData[1]].morphology['created']):
                    patient.hospitalStay[patientData[1]].morphology['created'] = patientData[11]
                    patient.hospitalStay[patientData[1]].morphology['value'] = patientData[13]
                    morfTests = parseStringTestsResult(patientData[13], 'mor_')
                    for morKey, morValue in morfTests.items():
                        subjectStudy = SubjectStudy()
                        subjectStudy.key = morKey
                        subjectStudy.type = patientData[4]
                        subjectStudy.created = patientData[11]
                        subjectStudy.full = morValue
                        splitedValue = morValue.split("[")
                        if len(splitedValue) > 1:
                            subjectStudy.value = splitedValue[0]
                        else:
                            subjectStudy.value = morValue

                        if morValue.find("[") > 0 and morValue.find("]") > 0:
                            subjectStudy.unit = morValue[morValue.find("[") + 1:morValue.find("]")]
                        else:
                            subjectStudy.unit = ''
                        patient.hospitalStay[patientData[1]].subjectStudies[subjectStudy.key] = subjectStudy
            else:
                subjectStudy = SubjectStudy()
                subjectStudy.key = urlify(patientData[12])
                subjectStudy.type = patientData[4]
                subjectStudy.created = patientData[11]
                if not pd.isnull(patientData[13]) and is_not_blank(patientData[13]):
                    subjectStudy.value = patientData[13].split(":")[1].split("[")[0]
                    subjectStudy.unit = patientData[13][patientData[13].find("[") + 1:patientData[13].find("]")]
                    subjectStudy.full = patientData[13]
                if subjectStudy.key not in patient.hospitalStay[patientData[1]].subjectStudies:
                    patient.hospitalStay[patientData[1]].subjectStudies[subjectStudy.key] = subjectStudy
                elif parseDateTime(subjectStudy.created) > parseDateTime(
                        patient.hospitalStay[patientData[1]].subjectStudies[subjectStudy.key].created):
                    patient.hospitalStay[patientData[1]].subjectStudies[subjectStudy.key] = subjectStudy

            # if

    elif patientData[4] == 'Dana medyczna opisowa':
        descriptiveMedicalData = DescriptiveMedicalData()
        descriptiveMedicalData.key = patientData[12]
        descriptiveMedicalData.value = patientData[13]
        patient.hospitalStay[patientData[1]].descriptiveMedicalDatas[urlify(patientData[12])] = patientData[13];


def buildPatientsArray(patients: dict):
    patientsList = []
    for id, patient in patients.items():
        output = builPatientSeries(patient)
        for item in output:
            patientsList.append(pd.Series(item))
        # patientsDf = patientsDf.append(builPatientSeries(patient), ignore_index=True)
    return patientsList


# TO DO - nadpisane wartosci powinno zwracac tablice seri
def builPatientSeries(patient: Patient):
    output = []
    for id, hospitalStay in patient.hospitalStay.items():
        patientArray = {'id': patient.id, 'age': patient.age, 'sex': patient.sex}
        for subjectStudyName in hospitalStay.subjectStudies:
            patientArray[subjectStudyName] = hospitalStay.subjectStudies[subjectStudyName].value
        for diagnosticTest in hospitalStay.diagnosticTests:
            patientArray[diagnosticTest.key] = diagnosticTest.value
        for descriptiveMedicalDataKey in hospitalStay.descriptiveMedicalDatas:
            if descriptiveMedicalDataKey in hospitalStay.descriptiveMedicalDatas:
                patientArray[descriptiveMedicalDataKey] = hospitalStay.descriptiveMedicalDatas[
                    descriptiveMedicalDataKey]
            else:
                patientArray[descriptiveMedicalDataKey] = ''
#        for icd9 in hospitalStay.icd9:
#            patientArray[icd9] = 1
        for icd10 in hospitalStay.icd10:
            patientArray[icd10] = 1
        output.append(patientArray)

    return output

def computeMedicineGroup(medicines, conn):
    output = {}
    medicinesSplited = medicines.split(",")
    for item in medicinesSplited:
        item = item.strip()
        rows = findByName(conn, item)
        if len(rows) > 0:
            output[item] = rows[0][3]
        else:
            output[item] = ''
    return output
        
#>'Barwa:jasnożółta<|>Przej:zupełna<|>SG:1.005<|>pH:8.0<|>LEU:nieobecne[ul]<|>NIT:nieobecne[ul]<|>mKET:nieobecne[mg/dl]<|>mURO:niewzmożony<|>BIL:nieobecna<|>ERY:nieobecne[ul]<|>GLU-mocz:nieobecna[mg/dl]<|>UPROT:nieobecne[mg/dl]'
def parseStringTestsResult(testResult: str, prefix: str):
    output = {}
    outputKey = []
    testResultSplited = testResult.split("<|>")
    
    for item in testResultSplited:
        splitedItem = item.split(":")
        key = urlify(prefix + splitedItem[0])
        if len(splitedItem) > 1:
            output[key] = splitedItem[1]
        else:
            output[key] = splitedItem[0]
        #outputKey.append(splitedItem[0])
    return output

def urlify(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)

    # Replace all runs of whitespace with a single dash
    s = re.sub(r"\s+", '-', s)
    
    s = unidecode(s)
    
    s = s.lower()
    
    return s

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

def findByName(conn, name):
    cur = conn.cursor()
    
    cur.execute("select * from doktop_descs where desc_id = (select desc_id from doktop_descs where name like ? limit 1) and col_idx = 12", (name,))
     
        
    rows = cur.fetchall()
    return rows;
#    for row in rows:
#        print(row)

def getSheetNames(excelFilePath: str):
    xl = pd.ExcelFile(excelFilePath)
    return xl.sheet_names

def transformDataFromFilesToDict(files: dict):
    existingPatients = {}
    for key, item in files.items():
        readPatientsFromFile(item['path'], item['sheetName'], existingPatients)
        print("Len: {}".format(len(existingPatients)))
    
    return existingPatients

def transformDataFromFilesToList(files: dict):
    existingPatients = transformDataFromFilesToDict(files)
    
    return buildPatientsArray(existingPatients)

In [15]:
files = {
    '2016': {
        'path': '/home/jovyan/work/data-to-import/mediplanner_2016.xls',
        'sheetName': 'mediplanner_2016.csv'
    },
    '2017': {
        'path': '/home/jovyan/work/data-to-import/mediplanner_2017.xlsx',
        'sheetName': 'mediplanner_2017.csv'
    },
    '2018': {
        'path': '/home/jovyan/work/data-to-import/mediplanner_2018.xlsx',
        'sheetName': 'mediplanner_2018_2.csv'
    },
    '2019': {
        'path': '/home/jovyan/work/data-to-import/mediplanner_2019.xlsx',
        'sheetName': 'mediplanner_2019.csv'
    }
}
existingPatients = transformDataFromFilesToDict(files)
patientsList = buildPatientsArray(existingPatients)
display(len(patientsList))

Len: 1190
Len: 2724
Len: 4333
Len: 5795


10500

In [16]:
patientsDf = pd.DataFrame()
patientsDf = patientsDf.append(patientsList, ignore_index=True)

display(patientsDf.shape)

(10500, 1457)

In [17]:
print(patientsDf.describe().T)
print("Cech występujących jedynie raz:")
patientsDf.describe().T[patientsDf.describe().T['count'] == 1.0].sum()

         count           mean            std    min       25%        50%  \
id     10500.0  950283.480286  546336.997879  102.0  324493.0  1257510.0   
Q25.2     78.0       1.000000       0.000000    1.0       1.0        1.0   
Q21.0   1508.0       1.000000       0.000000    1.0       1.0        1.0   
Q21.1   1491.0       1.000000       0.000000    1.0       1.0        1.0   
Z98.8   3985.0       1.000000       0.000000    1.0       1.0        1.0   
Q24.4     77.0       1.000000       0.000000    1.0       1.0        1.0   
Z95.8   1180.0       1.000000       0.000000    1.0       1.0        1.0   
Z95.2    144.0       1.000000       0.000000    1.0       1.0        1.0   
Q23.4   1019.0       1.000000       0.000000    1.0       1.0        1.0   
R23.0    991.0       1.000000       0.000000    1.0       1.0        1.0   
I50.0    288.0       1.000000       0.000000    1.0       1.0        1.0   
Q23.0    602.0       1.000000       0.000000    1.0       1.0        1.0   
Q23.3    159

count    273.0
mean     273.0
std        0.0
min      273.0
25%      273.0
50%      273.0
75%      273.0
max      273.0
dtype: float64

In [6]:
from pymongo import MongoClient
from urllib.parse import quote_plus
from bson import BSON
import bson
from io import StringIO

mongoUsername = quote_plus('mediplanner')
mongoPassword = quote_plus('PASS')
mongoHost = '54.38.129.3'
mongoPort = 27017
mongoDB = 'mediplanner'
mongoURI = "mongodb://{username}:{password}@{host}:{port}/{db}".format(
                    username = mongoUsername, password = mongoPassword,
                    host = mongoHost, port = mongoPort,
                    db = mongoDB
                )
client = MongoClient(mongoURI, connect=False)

In [13]:
#patients = client.mediplanner.patients
#patientIo = StringIO(toJson(existingPatients[181107]))
#patientSON = bson.SON(json.load(patientIo))
#patients.insert_one(patientSON)

In [21]:
count = 0
patients = client.mediplanner.patients
for key, patient in existingPatients.items():
    patientIo = StringIO(toJson(patient))
    patientSON = bson.SON(json.load(patientIo))
    patients.insert_one(patientSON)
    count = count + 1
print('Imported: ' + str(count))

Imported: 5795
